In [2]:
from random import *
from math import *
from array import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from mpl_toolkits.mplot3d import Axes3D

In [3]:
def pois(S):
    if S <= 100:
        temp = -1*S
        L = exp(temp)
        k = 0
        p = 1
        while p > L:
            k = k + 1
            p = p * random()
        p = k - 1
    else:
        p = floor(S + sqrt(S) * random())
    return int(p)

In [4]:
# Haversian distance function
def getDistanceFromLatLonInKm(lat1,lon1,lat2,lon2):
    R = 6371 # Radius of the earth in km
    dLat = deg2rad(lat2-lat1)  # deg2rad below
    dLon = deg2rad(lon2-lon1)
    a = sin(dLat/2) * sin(dLat/2) + cos(deg2rad(lat1)) * cos(deg2rad(lat2)) * sin(dLon/2) * sin(dLon/2)
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    d = R * c # Distance in km
    return d;

def deg2rad(deg):
    return deg * (pi/180)

In [14]:
data = pd.read_csv('/la_text_data_time.csv')
data = data[['CRIMETYPE', 'X', 'Y', 'time']]
data = data.sort_values(by = ['time'])
data = data.reset_index(drop = True)
eventLs = list(set(data['CRIMETYPE']))

In [27]:
# locate index of cell for this event
def clocate(x,y,minlat,minlong):
    return int(ceil(distance.euclidean(x, minlat)/1150)-1), int(ceil(distance.euclidean(y, minlon)/1000)-1)
def glocate(x,y,minlat,minlong):
    x_km = getDistanceFromLatLonInKm(x, y, minlat, y)
    y_km = getDistanceFromLatLonInKm(x, y, x, minlong)
    return int(ceil(x_km-1)), int(ceil(y_km-1))
# updates over every cell
def global_update(w, time_all_1,time_all_0, Ajk, mu_jk, theta):
    Ajk = Ajk * np.exp(-1*w*(time_all_1-time_all_0)) 
    Lam_jk = mu_jk + theta*w*Ajk
    return Ajk, Lam_jk
# updates for a single cell
def event_update(mu_g, theta, w, A_g, Lam_g, time_all, time_prev,i):
    dt = 0.00025
    mu_g = mu_g + dt * (mu_g/Lam_g - mu_g*(time_all[i]-time_prev))
    theta = theta + dt * (A_g/Lam_g - theta)
    A_g = A_g + 1
    Lam_g = mu_g + theta*w*A_g
    return mu_g, theta, A_g, Lam_g

In [53]:
import time

minlat=min(data['X'])
minlong=min(data['Y'])

#N = len(data)
N = 10000
w = 3
nx = 143
ny = 230
K = len(eventLs)
t = [0] * nx
mu = [0] * nx
A = [0] * nx
Lambda = [0]*nx
for i in range(nx):
    t[i] = [0] * ny
    mu[i] = [0] * ny
    A[i] = [0] * ny
    Lambda[i] = [0]*ny
Theta = [0] * N
Theta[0] = 3
for i in range(nx):
    A[i] = [1]*ny
    mu[i] = [0.001]*ny
tls = {eventLs[k]: t for k in range(K)}
Als = {eventLs[k]: A for k in range(K)}
muls = {eventLs[k]: mu for k in range(K)}
Lambdals = {eventLs[k]: Lambda for k in range(K)}

for i in range (1,N):    
    eventType = data['CRIMETYPE'][i]
    t0=time.time()
    [gx, gy] = clocate(data['X'][i],data['Y'][i],minlat,minlong)
    print time.time()-t0, "first"
    # overall update
    
    t0=time.time()
    for j in range(nx):
        for k in range(ny):
            [Als[eventType][j][k], Lambdals[eventType][j][k]]  = global_update(w, data['time'][i],data['time'][i-1], Als[eventType][j][k], muls[eventType][j][k], Theta[i-1])
    # event cell update
    print time.time()-t0, "sec"
    
    t0=time.time()
    [muls[eventType][gx][gy], Theta[i], Als[eventType][gx][gy], Lambdals[eventType][gx][gy]] = event_update(muls[eventType][gx][gy], Theta[i-1], w, Als[eventType][gx][gy], Lambdals[eventType][gx][gy], data['time'][range(i+1)], tls[eventType][gx][gy],i)
    print time.time()-t0, "third"
    # keep track of the previous time for each event type 
    tls[eventType][gx][gy] = data['time'][i]



0.00257396697998 first
2.24679803848 sec
0.00988793373108 third
0.000553846359253 first
1.88939189911 sec
0.00934791564941 third
0.000365972518921 first
2.33616781235 sec
0.00938200950623 third
0.00082802772522 first
2.24460291862 sec
0.000850915908813 third
0.000293016433716 first
2.20815420151 sec
0.00116300582886 third
0.000332832336426 first
2.62010788918 sec
0.00937819480896 third
0.000230073928833 first
2.66310691833 sec
0.0103402137756 third
0.00023078918457 first
2.34548807144 sec
0.00103807449341 third
0.000723123550415 first
2.4533200264 sec
0.00134706497192 third
0.000507116317749 first
2.2158510685 sec
0.00101089477539 third
0.000442028045654 first
2.82921910286 sec
0.00156903266907 third
0.00034499168396 first


KeyboardInterrupt: 

In [51]:
[gx, gy] = clocate(data['X'][i],data['Y'][i],minlat,minlong)

print gx
print gy

102
196
